###Silver Layer Script

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
my_service_credential = dbutils.secrets.get(scope= 'service_credential', key = 'app-reg-proj2') 

spark.conf.set("fs.azure.account.auth.type.azureproj2storage.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.azureproj2storage.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.azureproj2storage.dfs.core.windows.net", "6a66ad3e-9753-40ea-afd5-64662422bff1")
spark.conf.set("fs.azure.account.oauth2.client.secret.azureproj2storage.dfs.core.windows.net", my_service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.azureproj2storage.dfs.core.windows.net", "https://login.microsoftonline.com/721593f8-7174-49c9-9958-85c4489c203a/oauth2/token")


In [0]:
df_cal = spark.read.csv('abfss://bronze@azureproj2storage.dfs.core.windows.net/AdventureWorks_Calendar', inferSchema = True, header = True)


In [0]:
df_cus= spark.read.csv('abfss://bronze@azureproj2storage.dfs.core.windows.net/AdventureWorks_Customers',inferSchema = True, header = True)

In [0]:
df_prodCat = spark.read.csv('abfss://bronze@azureproj2storage.dfs.core.windows.net/AdventureWorks_Product_Categories', inferSchema = True, header = True)


In [0]:
df_prodSubCat = spark.read.csv('abfss://bronze@azureproj2storage.dfs.core.windows.net/AdventureWorks_Product_Subcategories', inferSchema = True, header = True)

In [0]:
df_prod = spark.read.csv('abfss://bronze@azureproj2storage.dfs.core.windows.net/AdventureWorks_Products', inferSchema = True, header = True)

In [0]:
df_returns = spark.read.csv('abfss://bronze@azureproj2storage.dfs.core.windows.net/AdventureWorks_Returns', inferSchema = True, header = True)

In [0]:
df_sales = spark.read.csv('abfss://bronze@azureproj2storage.dfs.core.windows.net/AdventureWorks_Sales*', inferSchema = True, header = True)

In [0]:
df_territories = spark.read.csv('abfss://bronze@azureproj2storage.dfs.core.windows.net/AdventureWorks_Territories*', inferSchema = True, header = True)

In [0]:

df_cal =df_cal.withColumn('Month', monthname('Date'))\
       .withColumn('Year', year('Date'))

df_cal.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_Calendar')


In [0]:
df_cus= df_cus.withColumn('FullName',concat_ws(' ',col('Prefix'),col('FirstName'),col('LastName')))
df_cus.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_Customers')

In [0]:
df_prod = spark.createDataFrame([], df_prod.schema).withColumn('ProductSKU', split(col('ProductSKU'),'-')[0])\
    .withColumn('ProductName',split(col('ProductName'), ' ')[0])
df_prod.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_Products')


In [0]:
df_sales = df_sales.withColumn('StockDate', to_timestamp('StockDate'))\
    .withColumn('OrderNumber', regexp_replace(col('OrderNumber'),'S','T'))

In [0]:
df_total_orders = df_sales.groupBy('OrderDate').agg(count('OrderNumber').alias('Total_orders'))

In [0]:
df_sales.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_Sales')

In [0]:
df_total_orders.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_Total_orders')

In [0]:
df_prodCat.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_Productcategories')

In [0]:
df_prodSubCat.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_Product_Subcategories')

In [0]:
df_returns.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_returns')

In [0]:
df_territories.write.mode('overwrite').save('abfss://silver@azureproj2storage.dfs.core.windows.net/AdventureWorks_Territories')